1. Gmailからの(指定の)受信メールを取得しデータフレームにする<br>
  [参考にしたサイト](https://di-acc2.com/system/rpa/3031/#index_id12)
2. データフレームを加工する

In [ ]:
""" ライブラリ読込 """
import imapclient
from backports import ssl
from OpenSSL import SSL 
import pyzmail
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
import os

""" (要編集) 引数指定 """ 
#ログイン情報
my_mail = os.getenv("my_mail")
app_password = os.getenv("app_password")

#メール検索条件
FolderName = "INBOX"
Search_KWD = ["FROM", "1695kose@wits2003.jp"]

""" 関数定義 """
def Get_Mail(my_mail, app_password, FolderName, Search_KWD):
    
    """ ① IMAPサーバー接続 & SSL化 """
    context = ssl.SSLContext(SSL.TLSv1_2_METHOD)
    imap = imapclient.IMAPClient("imap.gmail.com", ssl=True, ssl_context=context)
    
    """ ② IMAPログイン """
    imap.login(my_mail, app_password)
    
    
    """ ③ メール検索 """
    #対象の受信メールフォルダを指定
    imap.select_folder(FolderName, readonly=True)
    
    #検索キーワードを設定 & 検索キーワードに紐づくメールID検索
    KWD = imap.search(Search_KWD)
    
    #メールID→メール本文取得
    raw_message = imap.fetch(KWD,["BODY[]"])
    
    """  ④ 解析結果保存 """
    #検索結果保存用
    From_list     = []
    Cc_list       = []
    Bcc_list      = []
    Subject_list  = []
    Body_list     = []
    
    #メール解析
    for j in range(len(KWD)):
        #特定メール取得
        message = pyzmail.PyzMessage.factory(raw_message[KWD[j]][b"BODY[]"])

        #宛先取得
        From = message.get_addresses("from")
        From_list.append(From)

        Cc = message.get_addresses("cc")
        Cc_list.append(Cc)

        Bcc = message.get_addresses("bcc")
        Bcc_list.append(Bcc)

        #件名取得
        Subject = message.get_subject()
        Subject_list.append(Subject)

        #本文
        # "ignore"オプションをつけてdecodeすると、デコード不可な文字は無視して無理やりdecodeしてくれます。
        Body = message.text_part.get_payload().decode("iso-2022-jp", "ignore")
        Body_list.append(Body)


    #Pandas データフレーム
    table = pd.DataFrame({"From":From_list,
                          "Cc":Cc_list,
                          "Bcc":Bcc_list,
                          "件名":Subject_list,
                          "本文":Body_list,
                         })

    #IMAPログアウト
    # imap.logout()

    return table

In [ ]:
# 関数実行
mail_df = Get_Mail(my_mail, app_password, FolderName, Search_KWD)

In [ ]:
mail_df.head()

# データフレームの加工

In [ ]:
import numpy as np

In [ ]:
# 授業日報だけのメールを取得する
report_df = mail_df[mail_df['件名'].str.contains('授業日報')]

In [ ]:
# ●授業日　03月02日 (水)
# ●時間　20時00分から80分
# ●時間　20時00分から80分
# ●教科　

In [ ]:
# 日付だけ、時間だけのカラムを作る
report_df['日付'] = report_df['本文'].apply(lambda x : x[x.find('●授業日')+5:x.find('●時間')-2])
report_df['時間'] = report_df['本文'].apply(lambda x : x[x.find('●時間')+4:x.find('●教科')-2])
# report_df['日時'] = report_df.apply(lambda x: f"{x['日付']} {x['時間']}", axis=1)
report_df.head()
# 時間だけのカラムを作る

In [ ]:
# 講師だけのカラムを作る
report_df['講師'] = report_df['本文'].apply(lambda x : x[x.find('講師')+3:x.find('\r\n●授業日')])

In [ ]:
# 授業の様子(講師より)だけのカラムを作る
report_df['授業の様子'] = report_df['本文'].apply(lambda x : x[x.find('授業の様子(講師より)')+13:x.find('●本日')])
# 必要な列だけのdfにする
to_souma_df=report_df.loc[:, ['日付','時間', '講師','授業の様子']]
#文字コードをutf8に統一する為に、encoding="utf8"というオプションを記載
to_souma_df.to_csv("to_souma.csv",index=None,encoding="utf8")